-Simulate the progress of generation of predctions on the machine and make comparasion between the results.
- Generate the predictions in Batch Mode and compare them with the correspondings of sequential mode 

Load Libraries

In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
from keras.models import Model, load_model
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta
import plotly.io as pio

Initialize Tensorflow

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

DB Config

In [ ]:
#####DB Config
#####DB Config
host= "m53763edge"
port=27017
collection= "thermal_data"
all_data= True
get_preprocessed_data= False

Set parameters

In [ ]:
version= 'v0.0.7'
workspace= "G:\\Innovations@HELLER\\DN\\KI\\Zollern-FH-MillTrunMotor\\Datasets\\workspace\\data\\training"
assests_dir= 'G:\\Innovations@HELLER\\DN\\KI\\Zollern-FH-MillTrunMotor\\Datasets\\workspace\\assets\\'+version+'\\'
preprocessor_dir=assests_dir
preprocessor_name='preprocessor.p'
model_dir= assests_dir
model_name= 'model.h5'
###['Drehzahl','Temp_Motor','Temp_Klemmung', 'Temp_Lager']
model_input_Columns=  ['T_LAGER', 'T_MOTOR', 'T_KLEMMUNG']
output_variable= 'prediction'
prediction_variable= 'prediction'
window=20
time_shift= +2
response_time= 5
time_relevance= 5
recording_time= 5

Remove t_bett function

In [ ]:
def remove_t_bett(data: np.ndarray):
    result= data[:, 1:]
    result[:,0]= result[:,0] - data[:,0]
    result[:,1]= result[:,1] - data[:,0]
    return result

In [ ]:
def rounding(signals: np.ndarray):
    return np.round(signals,2)

In [ ]:
# ### Auxiliary function for rounding
# def rounding(signals: np.ndarray):
#     ##rounding t_motor to nearest integer
#     signals[:,1]= np.round(signals[:,1])
#     signals= np.round(signals, 2)
#     return signals

Load Model and Preprocessor

In [ ]:
### load models
pipeline= load(str(preprocessor_dir+preprocessor_name))
print('preprocessor is loaded successfully')
model= load_model(str(model_dir + model_name), compile= False)
print('Model is loaded successfully')
print (model.summary())

Set time priod needed

In [ ]:
start= datetime.strptime("2023-06-19T14:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end= datetime.strptime ("2023-06-20T14:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
# start= datetime.strptime("2023-06-15T14:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
# end= datetime.strptime ("2023-06-16T13:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")

Get Data from database

In [ ]:
## Read raw and preprocessing data
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db[collection].find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None
i=0
last_prediction= -500
prev_time= None
current_time= None
stop= False
### Go through events in DB
for event in event_list:
    i+=1
    record_list= event['content']
    keys= None
    ### Go through records in each event
    for record in record_list:
        ### Get right data between start / end 
        if record['date']< start or record['date'] > end:
            continue
        current_time= record['date']+ timedelta(hours= time_shift)
        if prev_time is not None  and (current_time -prev_time).total_seconds()> 5.5:
            print('Period= ', (current_time -prev_time).total_seconds())
            print('prev_time', prev_time)
            print('current_time', current_time)
            print('#########################################################')
        for item in record['raw_data'].keys():
            record['raw_data'][item]= [np.double(np.round(record['raw_data'][item],decimals=2))]
        record['raw_data']['date']=[current_time]
        record['raw_data']['given2model']=[record['given2model']]
        record['raw_data']['prediction']= [record['prediction']]
        ####################################
        #print ('After ',record)
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals= lf_signal_point
        else:
            signals= signals.append(lf_signal_point,ignore_index= True)
        prev_time= current_time
signals.reset_index(inplace= True)
output=  signals[output_variable]
machine_prediction= signals[prediction_variable]

Visualize Retreived Signals

In [ ]:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
selected_Columns= ['DRZ','Temp_Klemmung', 'Temp_Lager','Temp_Motor','Analytic_sol', prediction_variable, output_variable, 'db_prediction_abs_error']
y_axis_names= selected_Columns
#print(preds)
fig= make_subplots(rows=5,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
##DRZ
fig.add_trace(go.Scatter(x= signals['date'], y= signals['DRZ5'], name='DRZ', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'DRZ (RPM)', row= 1, col= 1)
##T_KLEMMUNG
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_KLEMMUNG'], name='T_KLEMMUNG', mode= scatter_mode), row= 2, col= 1)
fig.update_yaxes(title_text= 'T_KLEMMUNG C°', row= 2, col= 1)
##T_LAGER
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_LAGER'], name='T_LAGER', mode= scatter_mode), row= 3, col= 1)
fig.update_yaxes(title_text= 'T_LAGER C°', row= 3, col= 1)
##'T_MOTOR'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_MOTOR'], name='T_MOTOR', mode= scatter_mode), row= 4, col= 1)
fig.update_yaxes(title_text= 'T_MOTOR C°', row= 4, col= 1)
##'prediction_variable'
fig.add_trace(go.Scatter(x= signals['date'], y= signals[prediction_variable], name=prediction_variable, mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals['Analytic_sol'], name='Analytic_sol', mode= scatter_mode), row= 5, col= 1)
fig.update_yaxes(title_text= 'Rotor Temperatur C°', row= 5, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data')
fig.show()

Calculate the error after the correction

In [ ]:
# signals['db_prediction_abs_error']= signals[prediction_variable] -signals[output_variable]

In [ ]:
signals.columns

Read Magent Temperature File

In [ ]:
file_path= "G:\\Innovations@HELLER\\DN\\KI\\Zollern-FH-MillTrunMotor\\Datasets\\rough_data\\Magnet_temperature\\2023_06_19_Magnet_Temperature_ISO_230.csv"
df= pd.read_csv(filepath_or_buffer=file_path,infer_datetime_format= True, parse_dates= True)
df['date']= pd.to_datetime(df['date'])+ pd.DateOffset(hours=1)

Visualize the retreived data

In [ ]:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
selected_Columns= ['DRZ','Temp_Klemmung', 'Temp_Lager','Temp_Motor','Analytic_sol', prediction_variable, output_variable, 'db_prediction_abs_error']
y_axis_names= selected_Columns
#print(preds)
fig= make_subplots(rows=5,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
##DRZ
fig.add_trace(go.Scatter(x= signals['date'], y= signals['DRZ5'], name='given2model', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'DRZ (RPM)', row= 1, col= 1)
##T_KLEMMUNG
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_KLEMMUNG'], name='T_KLEMMUNG', mode= scatter_mode), row= 2, col= 1)
fig.update_yaxes(title_text= 'T_KLEMMUNG C°', row= 2, col= 1)
##T_LAGER
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_LAGER'], name='T_LAGER', mode= scatter_mode), row= 3, col= 1)
fig.update_yaxes(title_text= 'T_LAGER C°', row= 3, col= 1)
##'T_MOTOR'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_MOTOR'], name='T_MOTOR', mode= scatter_mode), row= 4, col= 1)
fig.update_yaxes(title_text= 'T_MOTOR C°', row= 4, col= 1)
##'prediction_variable'
fig.add_trace(go.Scatter(x= df['date'], y= df['magnet_temperature'], name='magnet_temperature', mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals[prediction_variable], name=prediction_variable, mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals['Analytic_sol'], name='Analytic_sol', mode= scatter_mode), row= 5, col= 1)
fig.update_yaxes(title_text= 'Rotor Temperatur C°', row= 5, col= 1)
#Restfehler
#fig.add_trace(go.Scatter(x= signals['date'], y= signals['db_prediction_abs_error'], name=' Restfehler', mode= scatter_mode),  row= 6 , col= 1)
    ## Draw the tolerence +-5
#fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals['db_prediction_abs_error'],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
#fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals['db_prediction_abs_error'],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
#fig.update_yaxes(title_text= 'Fehler bei Rotor Temperatur C°', row= 6, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data')
fig.show()

In [ ]:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
selected_Columns= ['DRZ','Temp_Klemmung', 'Temp_Lager','Temp_Motor','Analytic_sol', prediction_variable, output_variable, 'db_prediction_abs_error']
y_axis_names= selected_Columns
#print(preds)
fig= make_subplots(rows=1,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x= df['date'], y= df['magnet_temperature'], name='magnet_temperature', mode= scatter_mode), row= 1, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals[prediction_variable], name="Data-Driven-Prediction", mode= scatter_mode), row= 1, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals['Analytic_sol'], name='Analytic_sol', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'Rotor Temperatur C°', row= 1, col= 1)
##Restfehler
#fig.add_trace(go.Scatter(x= signals['date'], y= signals['db_prediction_abs_error'], name=' Restfehler', mode= scatter_mode),  row= 6 , col= 1)
    ## Draw the tolerence +-5
#fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals['db_prediction_abs_error'],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
#fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals['db_prediction_abs_error'],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
#fig.update_yaxes(title_text= 'Fehler bei Rotor Temperatur C°', row= 6, col= 1)
##'given2model'
fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data')
fig.show()

In [ ]:
# file_path= "G:\\Innovations@HELLER\\DN\\KI\\Zollern-FH-MillTrunMotor\\Datasets\\rough_data\\Rotor_temperatur.csv"
# def dateparse (time_str):    
#     return pd.to_datetime(time_str)
# df= pd.read_csv(filepath_or_buffer=file_path,infer_datetime_format= True, parse_dates= True)
# print(df.info())
# df['date']= pd.to_datetime(df['date'])+ pd.DateOffset(hours=1)
# print(df.info())
# scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
# fig= make_subplots(rows=1,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
# ## Magnet_temperature  
# fig.add_trace(go.Scatter(x= df['date'], y= df['magnet_temperature'], name='magnet_temperature', mode= scatter_mode), row= 1, col= 1)
# fig.update_yaxes(title_text= 'magnet_temperature (C°)', row= 1, col= 1)
# fig.update_layout(height=1200, width=1400, title_text= 'Rotortemperatur')
# fig.show()

Initialize model input

In [ ]:
rough_signals= signals[model_input_Columns].to_numpy()

Simulate the prediction by passing values with 5 seconds

In [ ]:
def extract_time_relevance_array(input_array: np.ndarray, output_shape):
        input_time = input_array.shape[1]  # in case of 5 Seconds it will be 240 seconds
        output_time = output_shape[1]  # in case of 2 Minutes it will be 10
        step = int(input_time / output_time)
        result_array = np.zeros(output_shape)
        for i in range(0, output_time):
            #print('place in result: ', output_time - i - 1, ' place in original: ', input_time - i * step - 1)
            result_array[0, output_time - i - 1, :] = input_array[0, input_time - i * step - 1, :]
        return result_array

In [ ]:
#### Simulate the prediction by passing values with 5 seconds ########
num_model_features= 3
preds= []
input_data= signals.copy()
shape=(1,window,num_model_features)
output_sec_shape= (1, int(window*np.ceil((time_relevance/recording_time))), num_model_features)
print('Requested buffer shape to the model= {}', shape)
print('Orginal buffer shape to the model= {}', output_sec_shape)
first_run= True
inputsignal_list= model_input_Columns
last_prediction_time= None
out= 0
prev_record_time= None
for index, data_point in input_data.iterrows():
    if prev_record_time is not None and (data_point['date']  - prev_record_time).total_seconds() < recording_time:
        print('Escaped!')
        #preds.append(out)
        signals= signals[signals['date']!= data_point['date']]
        continue
    else:
        prev_record_time= data_point['date']
    #print ('index: ', index, ', Date: ', data_point['date'])
    new_signal_list = [float(data_point[sig]) for sig in inputsignal_list]
    #print ('New_signal_list: ', new_signal_list, ' timestamp: ', data_point['date'])
    rough_input_array= np.asarray([new_signal_list])
    signals_ = pipeline.transform(rough_input_array)#np.asarray([new_signal_list])#
    if first_run:
        #print ('index: ', index, ', Date: ', data_point['date'])
        #print('time_ diff= ', str((data_point['date'] - last_prediction_time).total_seconds()))
        # Auffüllen der Struktur - Bei erstem Start ist nur ein Datensatz vorhanden,
        # dieser wird mit np.full in alle kopiert
        hf_array= np.float32(np.full(output_sec_shape, signals_[0]))
        array= extract_time_relevance_array(hf_array,shape)
        #outputs = ort_sess.run(None, {'x': array})
        outputs =model.predict(array)
        #out = outputs[0][0][0] / 1000
        out = outputs[0][0]
        #print('Predciction First time. Input Buffer: ', array, 'prediction: ', out)
        preds.append(out)
        first_run = False
        last_prediction_time= data_point['date']
        #print('prediction_time: ', data_point['date'])
        continue
    elif (data_point['date'] - last_prediction_time).total_seconds()< response_time:
            #print('No prediction. Just shift')
            hf_array [0][:-1] = hf_array[0][1:]
            hf_array[0, -1, :] = signals_[0]
            preds.append(out)
            continue 
    else:
        #print ('index: ', index, ', Date: ', data_point['date'])
        #print('time_ diff= ', str((data_point['date'] - last_prediction_time).total_seconds()))
        #print("....Shifting the 5 seconds buffer and insert the new 5s-value")
        hf_array [0][:-1] = hf_array[0][1:]
        hf_array[0, -1, :] = signals_[0]
        array= extract_time_relevance_array(hf_array, shape)
        array = np.float32(array)
        ## Apply predicitions on 2 minutes buffer
        outputs =model.predict(array)
        #out = outputs[0][0][0] / 1000
        out = outputs[0][0]
        #print('Prediction Normal state. Input Buffer: ', array, 'prediction: ', out)
        preds.append(out)
        last_prediction_time= data_point['date']
        #print('prediction_time: ', data_point['date'])
    #signals['local_preds'][index]= out
    #print(signals.iloc[index])
    #print('#########################################################################')

Assign local prediction to the main dataframe

In [ ]:
signals['local_preds']= preds
local_prediction_error= signals[output_variable] - signals['local_preds']

In [ ]:
org_signals= signals.copy()

Visualize the results

In [ ]:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
selected_Columns= ['DRZ','Temp_Klemmung', 'Temp_Lager','Temp_Motor', prediction_variable, 'local_preds']
y_axis_names= selected_Columns
#print(preds)
fig= make_subplots(rows=5,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
##DRZ
fig.add_trace(go.Scatter(x= signals['date'], y= signals['DRZ5'], name='given2model', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'DRZ (RPM)', row= 1, col= 1)
##T_KLEMMUNG
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_KLEMMUNG'], name='T_KLEMMUNG', mode= scatter_mode), row= 2, col= 1)
fig.update_yaxes(title_text= 'T_KLEMMUNG C°', row= 2, col= 1)
##T_LAGER
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_LAGER'], name='T_LAGER', mode= scatter_mode), row= 3, col= 1)
fig.update_yaxes(title_text= 'T_LAGER C°', row= 3, col= 1)
##'T_MOTOR'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_MOTOR'], name='T_MOTOR', mode= scatter_mode), row= 4, col= 1)
fig.update_yaxes(title_text= 'T_MOTOR C°', row= 4, col= 1)
##'prediction_variable'
#fig.add_trace(go.Scatter(x= signals['date'], y= signals[prediction_variable], name=prediction_variable, mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y= df['magnet_temperature'], name='magnet_temperature', mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals['local_preds'], name='local_preds', mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals['prediction'], name='Machine_preds', mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals['Analytic_sol'], name='Analytic_sol', mode= scatter_mode), row= 5, col= 1)
fig.update_yaxes(title_text= 'Rotor Temperatur C°', row= 5, col= 1)
# ##Restfehler
# fig.add_trace(go.Scatter(x= signals['date'], y= signals['local_preds'] - df['magnet_temperature'], name=' Restfehler lokaler Vorhersage', mode= scatter_mode),  row= 6 , col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y= signals['prediction'] - df['magnet_temperature'], name=' Restfehler Maschinenvorhersage', mode= scatter_mode),  row= 6 , col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y= signals['Analytic_sol'] - df['magnet_temperature'], name=' Restfehler analytische Lösung', mode= scatter_mode),  row= 6 , col= 1)
#     ## Draw the tolerence +-5
# #fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals['db_prediction_abs_error'],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
# #fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals['db_prediction_abs_error'],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
# fig.update_yaxes(title_text= 'Fehler bei Rotor Temperatur C°', row= 6, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data')
fig.show()

In [ ]:
### window  represents time period by each entry in the buffer
### Shift represents the jump from value to next one in the buffer
### sampling rate 
def generateDataSource(signal_input=None, input_columns: list = [], output_length: int = 1, signal_output=None, window=1, shift=1, sample_rate=1):
    #subsequence_len= (window -1) *shift + 1
    subsequence_len= (window) *shift
    Signal_Length = signal_input.shape[0]
    num_samples = 1 + int((Signal_Length - subsequence_len) / sample_rate)
    x = np.zeros(shape=(num_samples, window, signal_input.shape[1]))
    y = np.zeros(shape=(num_samples, output_length, 1))
    for i in range(num_samples):
        x[i] = np.asarray([signal_input[i*sample_rate + j * shift] for j in range(0,window)])
        y[i] = signal_output[i*sample_rate + (window-1) * shift :i*sample_rate+ (window-1) * shift + output_length]

    return x, y

In [ ]:
shift=1
sample_rate=1
df_file = signals
rough_data= df_file[model_input_Columns]
target= df_file['local_preds']
transformed_data= pipeline.transform(rough_data)
partitions, target = generateDataSource(signal_input=transformed_data, input_columns=model_input_Columns, output_length=1, signal_output=target, window=window, shift=shift, sample_rate=sample_rate)
pred= model.predict(partitions)
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, print_grid=True,  vertical_spacing=0.02)
fig.add_trace(go.Line(y=target.flatten(), name='local_preds'), row=1, col=1)
fig.add_trace(go.Line(y=pred.flatten(), name='Magent_temp_Pred'), row=1, col=1)
fig.update_yaxes(title_text='Magnet Temp (C°)', row=1, col=1)
fig.add_trace(go.Line(y=target.flatten() - pred.flatten(), name='Prediction Error (True -Predicted)'), row=2, col=1)
fig.update_yaxes(title_text='Prediction Error (C°)', row=2, col=1)
fig.update_xaxes(title_text='Zeit 1 = 2 Sek', row=2, col=1)
fig.update_layout(height=900, width=900, title_text='Vorhersage der Magnet-Temp {file}'.format(file = 'local'))
fig.show()
#pio.write_image(fig, str(assests_dir+'{file}.jpg'.format(file= file)), format='jpg')

In [ ]:
# images_path='//Heller.biz/hnt/Steuerungstechnik/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/csvs/smoothed/AI_Model/Backup/Z_Welle/v2.7.0/five_seconds/'
# ## Comparasion between two sources:
# from turtle import color
# scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
# selected_Columns= ['t_bett','t_motor', 't_spindle' ,'M8', 'M121', 'M127', 'M7','given2model', prediction_variable]
# y_axis_names= selected_Columns
# #print(preds)
# fig= make_subplots(rows=len(selected_Columns)+1,cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_Columns, vertical_spacing=0.02)
# for i in range(len(selected_Columns)):
#     fig.add_trace(go.Scatter(x= signals['date'], y= signals[selected_Columns[i]], name=selected_Columns[i], mode= scatter_mode), row= i+1, col= 1,)
#     fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
# ##Draw the prediciton and the real values of displacement on Welle
# fig.add_trace(go.Scatter(x= signals['date'],y= signals[output_variable], name=output_variable, mode= scatter_mode),  row= len(selected_Columns) , col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y=  signals['local_preds'], name='Local predictions', mode= scatter_mode),  row= len(selected_Columns) , col= 1)
# fig.update_yaxes(title_text= output_variable, row= len(selected_Columns), col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y=   1000*local_prediction_error, name='New Model prediction Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
# ## Draw the tolerence +-5
# fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(local_prediction_error,5), name='+5 Max Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(local_prediction_error,-5), name='-5 Min Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)

# #fig.add_trace(go.Scatter(x= signals['date'], y=   1000*( np.abs(signals[output_variable] - signals[prediction_variable])), name='Old Model prediction Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
# fig.update_yaxes(title_text= 'Error (Micro-meter)', row= len(selected_Columns)+1, col= 1)
# fig.update_layout(height=1200, width=1400, title_text= 'Prediction Results')
# #pio.write_image(fig, str(images_path +'versuch_20_09_iso.png'), format='png')
#fig.show()
